In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('display.max_colwidth', 200)

In [2]:
def read_jsonl_list(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            line_data = json.loads(line.rstrip('\n|\r'))
            data.append(line_data)
    print(f'Loaded {len(data)} records from {input_path}')
    return data

In [3]:
dl = read_jsonl_list('../data/work/data_boolean_la_only.jsonl')

Loaded 3798 records from ../data/work/data_boolean_la_only.jsonl


In [4]:
df = pd.DataFrame(dl)

In [5]:
df

,question_text,boolean_answer,long_answer_html,example_id
0,does a nissan rogue have 4 wheel drive,YES,"<Table> <Tr> <Th colspan=""2""> First Generation </Th> </Tr> <Tr> <Td colspan=""2""> 2011 Nissan Rogue S </Td> </Tr> <Tr> <Th colspan=""2""> Overview </Th> </Tr> <Tr> <Th> Also called </Th> <Td> Nissan ...",1654053568091710543
1,is there tax on feminine products in florida,NO,"<P> In July 2016 , New York State exempted feminine hygiene products from taxation , reducing the state 's tax revenue by an estimated US $10 million annually . Connecticut and Illinois also remov...",1747998273171432516
2,is sicily the most populated region in italy,NO,"<P> Sicily is a melting pot of a variety of different cultures and ethnicities , including the original Italic people , the Phoenicians , Carthaginians , Greeks , Romans , Byzantines , Saracens , ...",-4085091680622865749
3,is season 7 the final season of once upon a time,YES,"<P> The storyline is softly rebooted with a main narrative led by an adult Henry Mills , set several years after last season 's events . In February 2018 , it was announced the seventh season woul...",-7164339370337095894
4,is puerto rico a united states of america state,NO,"<P> Puerto Rico ( Spanish for `` Rich Port '' ) , officially the Commonwealth of Puerto Rico ( Spanish : Estado Libre Asociado de Puerto Rico , lit . `` Free Associated State of Puerto Rico '' ) a...",-6947398545896683467
...,...,...,...,...
3793,can an arector pili muscle change the surface area of your skin,NO,"<P> Contraction of the muscles has a number of different purposes . Its principal function in the majority of mammals is to provide insulation : air becomes trapped between the erect hairs , helpi...",3057935938219145522
3794,are there diamond mines in the united states,YES,"<Ul> <Li> Crater of Diamonds State Park , Arkansas ( Former mine now a state park ) </Li> <Li> Kelsey Lake Diamond Mine , Colorado ( Former mine no longer in operation ) </Li> <Li> Jastro Winkle A...",9067232858126635273
3795,does the us army use the metric system,YES,"<P> Although customary units are used more often than metric units in the U.S. , the SI system is used extensively in some fields such as science , medicine , the military , automobile production ...",5102369897282418212
3796,do all angles on a triangle add up to 180,YES,"<P> In several geometries , a triangle has three vertices and three sides , where three angles of a triangle are formed at each vertex by a pair of adjacent sides . In a Euclidean space , the sum ...",2550950555769482148


In [26]:
df[['question_type']] = df['question_text'].apply(lambda x: x.split(' ')[0])
df

,question_text,boolean_answer,long_answer_html,example_id,question_type
0,does a nissan rogue have 4 wheel drive,YES,"<Table> <Tr> <Th colspan=""2""> First Generation </Th> </Tr> <Tr> <Td colspan=""2""> 2011 Nissan Rogue S </Td> </Tr> <Tr> <Th colspan=""2""> Overview </Th> </Tr> <Tr> <Th> Also called </Th> <Td> Nissan ...",1654053568091710543,does
1,is there tax on feminine products in florida,NO,"<P> In July 2016 , New York State exempted feminine hygiene products from taxation , reducing the state 's tax revenue by an estimated US $10 million annually . Connecticut and Illinois also remov...",1747998273171432516,is
2,is sicily the most populated region in italy,NO,"<P> Sicily is a melting pot of a variety of different cultures and ethnicities , including the original Italic people , the Phoenicians , Carthaginians , Greeks , Romans , Byzantines , Saracens , ...",-4085091680622865749,is
3,is season 7 the final season of once upon a time,YES,"<P> The storyline is softly rebooted with a main narrative led by an adult Henry Mills , set several years after last season 's events . In February 2018 , it was announced the seventh season woul...",-7164339370337095894,is
4,is puerto rico a united states of america state,NO,"<P> Puerto Rico ( Spanish for `` Rich Port '' ) , officially the Commonwealth of Puerto Rico ( Spanish : Estado Libre Asociado de Puerto Rico , lit . `` Free Associated State of Puerto Rico '' ) a...",-6947398545896683467,is
...,...,...,...,...,...
3793,can an arector pili muscle change the surface area of your skin,NO,"<P> Contraction of the muscles has a number of different purposes . Its principal function in the majority of mammals is to provide insulation : air becomes trapped between the erect hairs , helpi...",3057935938219145522,can
3794,are there diamond mines in the united states,YES,"<Ul> <Li> Crater of Diamonds State Park , Arkansas ( Former mine now a state park ) </Li> <Li> Kelsey Lake Diamond Mine , Colorado ( Former mine no longer in operation ) </Li> <Li> Jastro Winkle A...",9067232858126635273,are
3795,does the us army use the metric system,YES,"<P> Although customary units are used more often than metric units in the U.S. , the SI system is used extensively in some fields such as science , medicine , the military , automobile production ...",5102369897282418212,does
3796,do all angles on a triangle add up to 180,YES,"<P> In several geometries , a triangle has three vertices and three sides , where three angles of a triangle are formed at each vertex by a pair of adjacent sides . In a Euclidean space , the sum ...",2550950555769482148,do


In [39]:
df['question_type'].value_counts().head(8)

is      1409
does     388
can      386
do       369
are      244
did      169
has      126
was      112
Name: question_type, dtype: int64

In [14]:
df.groupby('boolean_answer').count()

,question_text,long_answer_html,example_id
boolean_answer,,,
NO,1439,1439,1439
YES,2359,2359,2359


In [15]:
df[['boolean_answer']].describe()

,boolean_answer
count,3798
unique,2
top,YES
freq,2359
